In [1]:
import mysql.connector as mysql
import gurobipy as gpy
import numpy as np
from math import sqrt

In [2]:
db_user = 'root'
db_pwd = 'root'
db_host = 'localhost'
db_name = 'set_cover'

cnx = mysql.connect(user=db_user, passwd=db_pwd, host=db_host, db=db_name)
cursor = cnx.cursor()
query = 'select * from demand; select * from tower;'
results = cursor.execute(query, multi=True)

count = 1
data = []
for result in results:
    data.append(result.fetchall())

cursor.close()
cnx.close()
for i in data:
    print(i)
demand = np.array(data[0])
tower = np.array(data[1])

[(41.0, 989.0), (243.0, 946.0), (120.0, 502.0), (150.0, 197.0), (455.0, 212.0), (909.0, 732.0), (943.0, 795.0), (175.0, 896.0), (391.0, 307.0), (822.0, 731.0), (266.0, 436.0), (512.0, 205.0), (225.0, 18.0), (494.0, 130.0), (462.0, 533.0), (449.0, 36.0), (709.0, 123.0), (117.0, 260.0), (633.0, 113.0), (478.0, 701.0), (804.0, 298.0), (353.0, 471.0), (318.0, 36.0), (76.0, 913.0), (883.0, 397.0), (443.0, 334.0), (848.0, 675.0), (82.0, 340.0), (909.0, 981.0), (131.0, 285.0), (368.0, 128.0), (717.0, 12.0), (894.0, 6.0), (387.0, 488.0), (941.0, 744.0), (920.0, 268.0), (977.0, 436.0), (681.0, 403.0), (156.0, 328.0), (920.0, 336.0), (953.0, 893.0), (338.0, 839.0), (413.0, 402.0), (256.0, 817.0), (879.0, 19.0), (739.0, 973.0), (273.0, 397.0), (385.0, 753.0), (965.0, 24.0), (792.0, 339.0), (353.0, 645.0), (645.0, 279.0), (889.0, 373.0), (928.0, 511.0), (26.0, 495.0), (998.0, 40.0), (757.0, 699.0), (644.0, 979.0), (896.0, 58.0), (591.0, 254.0), (713.0, 906.0), (819.0, 893.0), (327.0, 193.0), (76.0

In [3]:
#returns a list of caller indices that tower t covers
def coverage(tower, demand):
    ret = []
    for caller in range(len(demand)):
        dist = sqrt((demand[caller][0] - tower[0]) * (demand[caller][0] - tower[0]) + 
                    (demand[caller][1] - tower[1]) * (demand[caller][1] - tower[1]))
        if dist <= 100:
            ret.append(caller)
    return ret

In [4]:
covers = {}
for t in range(len(tower)):
    covers[t] = coverage(tower[t], demand)

In [5]:
num_towers = len(tower)
num_callers = len(demand)

m = gpy.Model('celltower')
# Binary decision variable for tower inclusion
# x = [m.addVar(vtype=gpy.GRB.BINARY, name=f'x_{i}') for i in range(num_towers)]
x = m.addVars(num_towers, vtype=gpy.GRB.BINARY, name='x')
# Binary decision variable for caller served
# y = [m.addVar(vtype=gpy.GRB.BINARY, name=f'y_{i}') for i in range(num_callers)]
y = m.addVars(num_callers, vtype=gpy.GRB.BINARY, name='y')
m.setParam('MIPGap', 0.07)
m.update()
# Objective: minimize number of towers used
m.setObjective(gpy.quicksum(x), gpy.GRB.MINIMIZE)
# Constraint: each caller must be served by at least 1 tower
m.addConstrs(y[i] == 1 for i in range(num_callers))
# Constraint: Each caller must be covered by at least 1 tower
m.addConstrs(gpy.quicksum(x[t] for t in range(num_towers) if r in covers[t]) >= y[r] 
              for r in range(num_callers))
# Constraint: caller must be within 100 units of at least 1 tower
# for i in demand:
#     m.addConstr(checkPointRange(min_tower, i))

m.update()
m.optimize()

for v in m.getVars():
    if v.x > 0.5 and v.varName[0]=='x':  
        print(f'{v.varName}: {v.x}')

print(f'Gurobi Model Status: {m.Status}')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-08
Set parameter MIPGap to value 0.07
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 4000 rows, 6489 columns and 260091 nonzeros
Model fingerprint: 0xc69d3a0a
Variable types: 0 continuous, 6489 integer (6489 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 63.0000000
Presolve removed 2467 rows and 3199 columns
Presolve time: 0.59s
Presolved: 1533 rows, 3290 columns, 175511 nonzeros
Found heuristic solution: objective 61.0000000
Variable types: 0 continuous, 3290 integer (3290 binary)

Root relaxation: objective 3.580329e+01, 8462 iterations, 2.28 seconds (4

Gurobi Model Status: 2
